# Dataproject of the employmentrate in the danish municipalities

In this dataproject we will describe and analyse the employment rate of the danish municipalities in the period 2008-2018. We will, among other things, analyse the correlation between the employment rate and disposable income in the municipalities together with the correlation between the employment rate and the accidents ratio.
Thus the goal of the project is to come up with different possible explanations to why the employment rate in the danish municipalities is different. We are aware that we cannot based on the analyse, say anything about the causality - to do that we need to perform regressions.
We start by importing and the needed packages; for then to turn to the cleaning proces of the datasets of the variables: Employmentrate, Accidents. 
Finally we will analyse the correlation mentioned above graphically.


1. **Importing needed packages**

***Note*** : Before procedeeing, make sure you have acces to the data used in this project; you will need to instal: pip install git+https://github.com/elben10/pydst in ypur Anaconda Prompt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import pandas as pd
import B_funktioner
import pydst as pydst
dst = pydst.Dst(lang='en')



This first section collects datasets from DST and cleans in order to get the wanted variables
Collection of datasets and cleaning. To get an overview of which variables are contained in each datasetWe we start by running the following lines separately. Every dataset is divided into the danish municipalities.

#Tænker at dette skal gøres mere udførligt osv. Det er bare lige for vores eget overbliks skyld
#Måske skal vi overveje at have helt i starten noget der undersøger hvilke datasæt vi vil hente ala nedenstående

In [8]:
#Running this line gives an overview of the datasets in: Statistics Denmark
dst.get_subjects()

#The dataset of the population size
dst.get_tables(subjects=['02']).head(10)
dst.get_variables(table_id = 'FOLK1A')

#The dataset of the employment rate
dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'RAS200')

#The dataset of the number of accidents
dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'UHELDK7')

#The dataset of immigrants
dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'FOLK1E')

#The dataset of the number of sentences
dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'STRAF41')

#The datset of the disposable income
dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id='INDKP107')




,id,desc,active,hasSubjects
0,02,Population and elections,True,True
1,05,Living conditions,True,True
2,03,Education and knowledge,True,True
3,18,Culture and National Church,True,True
4,04,"Labour, income and wealth",True,True
5,06,Prices and consumption,True,True
6,14,National accounts and government finances,True,True
7,16,Money and credit market,True,True
8,13,External economy,True,True
9,07,Business sector in general,True,True


In [11]:
#Downloading the population data of the danish municipalities (*note* the structure is almost identical for the ofther datasets, why the same code will not be explanied every time)

variables = {'OMRÅDE':['*'],'Tid':['*']} #Hvorfor kører vi denne først??? - den siger vel ikke noget?
#Downloading and defining the dataset for the population of the danish municipalities
pop = dst.get_data(table_id = 'FOLK1A', variables=variables)

#Using the functions made in [dataproject.py] to clean and sort the dataset
pop = B_funktioner.initial_rename(pop,'population')
pop = B_funktioner.only_keep_municipalities_and_years(pop,0,2008,2018)
pop = B_funktioner.sort_reset(pop)
print(pop)

# Deleting unnecessary colunms, which we are not going to use
del pop['KØN']
del pop['ALDER']
del pop['CIVILSTAND']
pop


municipality  year    KØN  ALDER CIVILSTAND  population
0        Aabenraa  2008  Total  Total      Total       60189
1        Aabenraa  2009  Total  Total      Total       60392
2        Aabenraa  2010  Total  Total      Total       59978
3        Aabenraa  2011  Total  Total      Total       59795
4        Aabenraa  2012  Total  Total      Total       59600
...           ...   ...    ...    ...        ...         ...
1084          Ærø  2014  Total  Total      Total        6393
1085          Ærø  2015  Total  Total      Total        6276
1086          Ærø  2016  Total  Total      Total        6290
1087          Ærø  2017  Total  Total      Total        6177
1088          Ærø  2018  Total  Total      Total        6178

[1089 rows x 6 columns]


,municipality,year,population
0,Aabenraa,2008,60189
1,Aabenraa,2009,60392
2,Aabenraa,2010,59978
3,Aabenraa,2011,59795
4,Aabenraa,2012,59600
...,...,...,...
1084,Ærø,2014,6393
1085,Ærø,2015,6276
1086,Ærø,2016,6290
1087,Ærø,2017,6177


In [12]:
#Downloading the employment data of the danish municipalities

variables={'OMRÅDE':['*'], 'BEREGNING':['*'], 'TID':['*']} #Igen hvad gør denne linje?? 
#downloading and defining the employment data of the danish municipalities
empl = dst.get_data(table_id = 'RAS200', variables=variables)

#Using the functions made in [dataproject.py]:
empl = B_funktioner.initial_rename(empl,'employment rate')
empl = B_funktioner.only_keep_municipalities_and_years(empl,0,2008,2018)
#empl = B_funktioner.row_chooser(empl,BEREGNING,'Employment rate')
#Hvorfor er den ovenfor med #?
print(empl)

#As we will only examine the employment rate we make use of the following for-loop to eliminate the rows containing data of the "Economic activity rate" 
for val in ['Employment rate']: 
    I = empl.BEREGNING.str.contains(val)
    empl = empl.loc[I == True] # Deleting everything else than the : "Employment rate"
print(empl) 

#Sorting the table:
empl= B_funktioner.sort_reset(empl)

#Skal nedenstående med????
#Deleting columns for which we will not examine.
#del empl['KØN']
#del empl['ALDER']
#del empl['HERKOMST']
#del empl['BEREGNING']


empl



municipality               BEREGNING  year HERKOMST       ALDER    KØN  \
0              Køge         Employment rate  2011    Total  Age, total  Total   
1              Køge         Employment rate  2012    Total  Age, total  Total   
2              Køge         Employment rate  2013    Total  Age, total  Total   
3              Køge         Employment rate  2014    Total  Age, total  Total   
4           Rødovre         Employment rate  2011    Total  Age, total  Total   
...             ...                     ...   ...      ...         ...    ...   
2547    Brønderslev  Economic activity rate  2018    Total  Age, total  Total   
2548      Silkeborg         Employment rate  2018    Total  Age, total  Total   
2549      Silkeborg  Economic activity rate  2018    Total  Age, total  Total   
2550  Frederikshavn         Employment rate  2018    Total  Age, total  Total   
2551  Frederikshavn  Economic activity rate  2018    Total  Age, total  Total   

      employment rate  
0         

,municipality,BEREGNING,year,HERKOMST,ALDER,KØN,employment rate
0,Aabenraa,Employment rate,2008,Total,"Age, total",Total,74.6
1,Aabenraa,Employment rate,2009,Total,"Age, total",Total,71.6
2,Aabenraa,Employment rate,2010,Total,"Age, total",Total,70.9
3,Aabenraa,Employment rate,2011,Total,"Age, total",Total,70.2
4,Aabenraa,Employment rate,2012,Total,"Age, total",Total,69.5
...,...,...,...,...,...,...,...
1084,Ærø,Employment rate,2014,Total,"Age, total",Total,65.1
1085,Ærø,Employment rate,2015,Total,"Age, total",Total,65.8
1086,Ærø,Employment rate,2016,Total,"Age, total",Total,67.0
1087,Ærø,Employment rate,2017,Total,"Age, total",Total,68.1


In [15]:

#Downloading accident data 

variables={'OMRÅDE':['*'], 'UHELDA':['*'], 'TID':['*']} #Igen hvorfor er denne med?

#Downloading and defining:
accidents = dst.get_data(table_id = 'UHELDK7', variables=variables)

#Using the functions made in [dataproject.py]:
accidents = B_funktioner.initial_rename(accidents,'Number of accidents')
accidents = B_funktioner.only_keep_municipalities_and_years(accidents,0,2008,2018)
#accidents = B_funktioner.row_chooser(accidents,UHELDA,'All accidents')

#The 2 lines below ensures, that we only are going to examine "All accidents" and not the subcategories.
I = accidents.UHELDA.str.contains('All accidents')
accidents = accidents.loc[I == True]

Sorting
accidents= B_funktioner.sort_reset(accidents)

#Skal nedenstående med????
#Deleting columns for which we will not examine.
#del accidents['UHELDA']
#del accidents['BYLAND']
#del accidents['UHELDSIT']


accidents



,municipality,UHELDA,year,BYLAND,UHELDSIT,Number of accidents
0,Aabenraa,All accidents,2008,Inside urban area,One-vehicle accidents,4
1,Aabenraa,All accidents,2009,Inside urban area,One-vehicle accidents,4
2,Aabenraa,All accidents,2010,Inside urban area,One-vehicle accidents,1
3,Aabenraa,All accidents,2011,Inside urban area,One-vehicle accidents,4
4,Aabenraa,All accidents,2012,Inside urban area,One-vehicle accidents,4
...,...,...,...,...,...,...
792,Ærø,All accidents,2009,Inside urban area,One-vehicle accidents,2
793,Ærø,All accidents,2011,Inside urban area,One-vehicle accidents,2
794,Ærø,All accidents,2015,Inside urban area,One-vehicle accidents,2
795,Ærø,All accidents,2016,Inside urban area,One-vehicle accidents,2


In [16]:
#Dowloading and defining the data on immigrants from non-western countries  in the danish municipalities

variables={'OMRÅDE':['*'],'HERKOMST':{'*'}, 'TID':['*']} #Igen hvorfor er denne med?
immigrants = dst.get_data(table_id = 'FOLK1E', variables=variables)

#Using the functions made in [dataproject.py]:
immigrants = B_funktioner.initial_rename(immigrants,'Immigrants')
immigrants = B_funktioner.only_keep_municipalities_and_years(immigrants,0,2008,2018)
#immigrants = B_funktioner.row_chooser(immigrants,HERKOMST,'Immigrants from non-western countries')

#The 2 lines below ensures, that we only are going to examine "immigrants from non-western countries"
I = immigrants.HERKOMST.str.contains('Immigrants from non-western countries')
immigrants = immigrants.loc[I == True]

#Mangler vi ikke:
#Deleting columns whcích we are not going to examine:
#del immigrants['KØN']
#del immigrants['ALDER']
#del immigrants['HERKOMST']


immigrants = B_funktioner.sort_reset(immigrants)
immigrants



,municipality,HERKOMST,year,KØN,ALDER,Immigrants
0,Aabenraa,Immigrants from non-western countries,2008,Total,Total,1787
1,Aabenraa,Immigrants from non-western countries,2009,Total,Total,1822
2,Aabenraa,Immigrants from non-western countries,2010,Total,Total,1817
3,Aabenraa,Immigrants from non-western countries,2011,Total,Total,1887
4,Aabenraa,Immigrants from non-western countries,2012,Total,Total,1906
...,...,...,...,...,...,...
1084,Ærø,Immigrants from non-western countries,2014,Total,Total,104
1085,Ærø,Immigrants from non-western countries,2015,Total,Total,103
1086,Ærø,Immigrants from non-western countries,2016,Total,Total,123
1087,Ærø,Immigrants from non-western countries,2017,Total,Total,145


In [20]:
#Dowloading data on sentences of the danish municipalities

variables={'OMRÅDE':['*'], 'TID':['*']} #Igen - hvorfor?
#Downloading and defining:
sentences = dst.get_data(table_id = 'STRAF41', variables=variables)

#Using the functions made in [dataproject.py]:
sentences = B_funktioner.initial_rename(sentences,'Number of sentences')
sentences = B_funktioner.only_keep_municipalities_and_years(sentences,0,2008,2018)
sentences = B_funktioner.sort_reset(sentences)

#Mangler vi ikke:
#Deleting columns whcích we are not going to examine:
#del sentences['KØN']
#del sentences['ALDER']
#del sentences['HERKOMST']

sentences


,municipality,year,ALDER,KØN,Number of sentences
0,Aabenraa,2008,"Age, total",Total,49197
1,Aabenraa,2009,"Age, total",Total,49132
2,Aabenraa,2010,"Age, total",Total,49145
3,Aabenraa,2011,"Age, total",Total,49146
4,Aabenraa,2012,"Age, total",Total,48994
...,...,...,...,...,...
1084,Ærø,2014,"Age, total",Total,5612
1085,Ærø,2015,"Age, total",Total,5561
1086,Ærø,2016,"Age, total",Total,5553
1087,Ærø,2017,"Age, total",Total,5489


In [29]:
#Dowloading data on average disposable income of the danish municipalities

variables={'OMRÅDE':['*'], 'TID':['*'], 'ENHED':['*']} #IGen hvorfor
avginc = dst.get_data(table_id = 'INDKP107', variables=variables)

#Using the functions made in [dataproject.py]:
avginc = B_funktioner.initial_rename(avginc,'Avg disp income')
avginc = B_funktioner.only_keep_municipalities_and_years(avginc,0,2008,2018)

#The 2 lines below ensures, that we only are going to examine "Average income for all people"
I = avginc.ENHED.str.contains('all people')
avginc = avginc.loc[I == True]

#Sorting
avginc = B_funktioner.sort_reset(avginc)

#Mangler vi ikke:
#Deleting columns whcích we are not going to examine:
#del avginv['ENHED']
#del avginv['KOEN']
#del avginv['UDDNIV']
#del avginv['INDKOMSTTYPE']

avginc


,municipality,year,ENHED,KOEN,UDDNIV,INDKOMSTTYPE,Avg disp income
0,Aabenraa,2008,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),125064
1,Aabenraa,2009,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),123277
2,Aabenraa,2010,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),130019
3,Aabenraa,2011,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),132330
4,Aabenraa,2012,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),133960
...,...,...,...,...,...,...,...
1064,Ærø,2014,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),132795
1065,Ærø,2015,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),129903
1066,Ærø,2016,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),136753
1067,Ærø,2017,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),142416


In [36]:
#Merge of all the downloaded datasets to one

Tabel1 = pop

Tabel1 = Tabel1.join([empl['employment rate'],accidents['Number of accidents'],immigrants['Immigrants'],sentences['Number of sentences'],avginc['Avg disp income']]) #['x'] tells which columns shall be used / transported to the new main tabel.

Tabel1




,municipality,year,population,employment rate,Number of accidents,Immigrants,Number of sentences,Avg disp income
0,Aabenraa,2008,60189,74.6,4.0,1787,49197,125064.0
1,Aabenraa,2009,60392,71.6,4.0,1822,49132,123277.0
2,Aabenraa,2010,59978,70.9,1.0,1817,49145,130019.0
3,Aabenraa,2011,59795,70.2,4.0,1887,49146,132330.0
4,Aabenraa,2012,59600,69.5,4.0,1906,48994,133960.0
...,...,...,...,...,...,...,...,...
1084,Ærø,2014,6393,65.1,NaN,104,5612,NaN
1085,Ærø,2015,6276,65.8,NaN,103,5561,NaN
1086,Ærø,2016,6290,67.0,NaN,123,5553,NaN
1087,Ærø,2017,6177,68.1,NaN,145,5489,NaN


2. **Analyze**


3. **Conclusion**